In [20]:
# Python script to insert station names from CSV filenames

import os
import sqlite3

# ✅ Updated absolute paths
csv_folder = r"C:\Users\unitf\OneDrive\Desktop\FYP\mrt_absa_webapp\mrt_reviews_csvs"
db_path = r"C:\Users\unitf\OneDrive\Desktop\FYP\mrt_absa_webapp\instance\mrt_reviews.db"

# Connect to SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Loop through CSV files and insert station names
for filename in os.listdir(csv_folder):
    if filename.endswith(".csv"):
        station_name = filename.replace(".csv", "").strip().upper()
        try:
            cursor.execute("INSERT OR IGNORE INTO stations (station_name) VALUES (?)", (station_name,))
            print(f"[ADDED] {station_name}")
        except Exception as e:
            print(f"[ERROR] Failed to add {station_name}: {e}")

conn.commit()
conn.close()
print("✅ All station names inserted.")


[ADDED] KG04 KWASA DAMANSARA
[ADDED] KG06 KOTA DAMANSARA
[ADDED] KG07 SURIAN
[ADDED] KG08 MUTIARA DAMANSARA
[ADDED] KG09 BANDAR UTAMA
[ADDED] KG10 TTDI
[ADDED] KG11 PHILEO DAMANSARA
[ADDED] KG13 PUSAT BANDAR DAMANSARA
[ADDED] KG14 SEMANTAN
[ADDED] KG15 MUZIUM NEGARA
[ADDED] KG16 PASAR SENI
[ADDED] KG17 MERDEKA
[ADDED] KG18A BUKIT BINTANG
[ADDED] KG20 TRX
[ADDED] KG21 COCHRANE
[ADDED] KG22 MALURI
[ADDED] KG23 TAMAN PERTAMA
[ADDED] KG24 TAMAN MIDAH
[ADDED] KG25 TAMAN MUTIARA
[ADDED] KG27 TAMAN SUNTEX
[ADDED] KG28 SRI RAYA
[ADDED] KG29 BTHO
[ADDED] KG30 BATU 11
[ADDED] KG31 BUKIT DUKONG
[ADDED] KG33 SUNGAI JERNIH
[ADDED] KG34 STADIUM KAJANG
[ADDED] KG35 KAJANG
✅ All station names inserted.


In [1]:
#insert reviews to reviews table 
import os
import sqlite3
import pandas as pd
from datetime import datetime

# File and DB paths
csv_folder = r"C:\Users\unitf\OneDrive\Desktop\FYP\mrt_absa_webapp\mrt_reviews_csvs"
db_path = r"C:\Users\unitf\OneDrive\Desktop\FYP\mrt_absa_webapp\instance\mrt_reviews.db"

# Connect to the SQLite database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get station_name → station_id map from stations table
cursor.execute("SELECT station_id, station_name FROM stations")
station_map = {name.strip().upper(): sid for sid, name in cursor.fetchall()}

# Process CSV files
for filename in os.listdir(csv_folder):
    if filename.endswith(".csv"):
        station_name_raw = filename.replace(".csv", "").strip().upper()
        station_id = station_map.get(station_name_raw)

        if not station_id:
            print(f"[WARNING] {station_name_raw} not found in stations table.")
            continue

        try:
            df = pd.read_csv(os.path.join(csv_folder, filename))
            if 'date' not in df.columns or 'cleaned_reviews' not in df.columns:
                print(f"[ERROR] Missing required columns in {filename}")
                continue

            for _, row in df.iterrows():
                review_date = str(row['date']).strip()
                review = str(row['cleaned_reviews']).strip()

                cursor.execute("""
                    INSERT INTO reviews (station_id, station_name, review_date, raw_reviews)
                    VALUES (?, ?, ?, ?)
                """, (station_id, station_name_raw, review_date, review))

            print(f"[SUCCESS] Imported reviews from {filename}")

        except Exception as e:
            print(f"[FAILED] Could not import {filename}: {e}")

conn.commit()
conn.close()
print("✅ All review data inserted into reviews table.")


OperationalError: no such table: stations

In [11]:
import os
import sqlite3
import pandas as pd

# === SETTINGS ===
csv_folder = r"C:\Users\unitf\OneDrive\Desktop\FYP\mrt_absa_webapp\mrt_reviews_csvs"
db_path = r"C:\Users\unitf\OneDrive\Desktop\FYP\mrt_absa_webapp\mrt_reviews.db"

# === CONNECT TO DATABASE ===
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# === LOAD STATION NAME TO ID MAPPING ===
cursor.execute("SELECT id, station_name FROM stations")
station_map = {name.strip().upper(): id for id, name in cursor.fetchall()}

# === PROCESS EACH CSV FILE ===
for filename in os.listdir(csv_folder):
    if filename.endswith(".csv"):
        filepath = os.path.join(csv_folder, filename)

        station_name_raw = filename.replace(".csv", "").strip().upper()
        station_id = station_map.get(station_name_raw)

        if station_id is None:
            print(f"[WARNING] Station '{station_name_raw}' not found in stations table. Skipping.")
            continue

        try:
            df = pd.read_csv(filepath)

            if "date" not in df.columns or "cleaned_reviews" not in df.columns:
                print(f"[ERROR] Missing required columns in: {filename}")
                continue

            for _, row in df.iterrows():
                review_date = row['date']
                review_text = row['cleaned_reviews']
                cursor.execute("""
                    INSERT INTO station_reviews (station_id, station_name, review_date, review)
                    VALUES (?, ?, ?, ?)
                """, (station_id, station_name_raw, review_date, review_text))

            print(f"[SUCCESS] Imported: {filename} -> Station ID {station_id}")
        except Exception as e:
            print(f"[FAILED] Error importing {filename}: {e}")

# === COMMIT & CLOSE ===
conn.commit()
conn.close()
print("✅ All CSVs imported to mrt_reviews.")
print("[INFO] Connecting to DB at:", db_path)



[FAILED] Error importing KG04 KWASA DAMANSARA.csv: no such table: station_reviews
[FAILED] Error importing KG06 KOTA DAMANSARA.csv: no such table: station_reviews
[FAILED] Error importing KG07 SURIAN.csv: no such table: station_reviews
[FAILED] Error importing KG08 MUTIARA DAMANSARA.csv: no such table: station_reviews
[FAILED] Error importing KG09 BANDAR UTAMA.csv: no such table: station_reviews
[FAILED] Error importing KG10 TTDI.csv: no such table: station_reviews
[FAILED] Error importing KG11 PHILEO DAMANSARA.csv: no such table: station_reviews
[FAILED] Error importing KG13 PUSAT BANDAR DAMANSARA.csv: no such table: station_reviews
[FAILED] Error importing KG14 SEMANTAN.csv: no such table: station_reviews
[FAILED] Error importing KG15 MUZIUM NEGARA.csv: no such table: station_reviews
[FAILED] Error importing KG16 PASAR SENI.csv: no such table: station_reviews
[FAILED] Error importing KG17 MERDEKA.csv: no such table: station_reviews
[FAILED] Error importing KG18A BUKIT BINTANG.csv: no

In [5]:

import os
import sqlite3
import pandas as pd

# === CONFIG ===
DATABASE_PATH = r"C:\Users\unitf\OneDrive\Desktop\FYP\mrt_absa_webapp\mrt_reviews.db  # change this to your actual SQLite DB path
CSV_FOLDER = r"C:\Users\unitf\OneDrive\Desktop\FYP\mrt_absa_webapp\mrt_reviews_csvs"

# === CONNECT TO DATABASE ===
conn = sqlite3.connect(DATABASE_PATH)
cursor = conn.cursor()

# === LOAD station_name TO station_id MAP ===
cursor.execute("SELECT id, station_name FROM stations")
station_map = {name.strip().upper(): id for id, name in cursor.fetchall()}

# === LOOP THROUGH EACH CSV FILE ===
for file_name in os.listdir(CSV_FOLDER):
    if not file_name.endswith(".csv"):
        continue

    station_name_raw = file_name.replace(".csv", "").strip().upper()
    station_id = station_map.get(station_name_raw)

    if station_id is None:
        print(f"⚠️ Station not found in DB: {station_name_raw}")
        continue

    full_path = os.path.join(CSV_FOLDER, file_name)
    df = pd.read_csv(full_path)

    # Rename for clarity
    df.columns = [c.lower().strip() for c in df.columns]
    if 'date' not in df.columns or 'cleaned_review' not in df.columns:
        print(f"❌ Skipping {file_name}: Missing required columns.")
        continue

    # Insert each row
    rows = [(station_id, station_name_raw, row['date'], row['cleaned_review']) for _, row in df.iterrows()]
    cursor.executemany("""
        INSERT INTO mrt_reviews (station_id, station_name, review_date, review)
        VALUES (?, ?, ?, ?)
    """, rows)

    print(f"✅ Inserted {len(rows)} rows from {file_name}")

# === COMMIT & CLOSE ===
conn.commit()
conn.close()
print("✅ All CSV files processed.")


SyntaxError: EOL while scanning string literal (3429854308.py, line 6)

In [5]:
#TO CLEAN ALL THE NULL VALUES IN EACH CSV MRT REVIEWS
import os
import pandas as pd

folder = r"C:\Users\unitf\OneDrive\Desktop\FYP\mrt_absa_webapp\mrt_reviews_csvs"

for file in os.listdir(folder):
    if file.endswith(".csv"):
        path = os.path.join(folder, file)
        try:
            df = pd.read_csv(path)
            if "cleaned_reviews" in df.columns:
                df = df.dropna(subset=['cleaned_reviews'])
                df = df[df['cleaned_reviews'].str.strip() != ""]
                df.to_csv(path, index=False)
                print(f"[CLEANED] {file}")
            else:
                print(f"[SKIPPED] {file} missing 'cleaned_reviews' column.")
        except Exception as e:
            print(f"[ERROR] {file}: {e}")


[CLEANED] KG04 KWASA DAMANSARA.csv
[CLEANED] KG06 KOTA DAMANSARA.csv
[CLEANED] KG07 SURIAN.csv
[CLEANED] KG08 MUTIARA DAMANSARA.csv
[CLEANED] KG09 BANDAR UTAMA.csv
[CLEANED] KG10 TTDI.csv
[CLEANED] KG11 PHILEO DAMANSARA.csv
[CLEANED] KG13 PUSAT BANDAR DAMANSARA.csv
[CLEANED] KG14 SEMANTAN.csv
[CLEANED] KG15 MUZIUM NEGARA.csv
[CLEANED] KG16 PASAR SENI.csv
[CLEANED] KG17 MERDEKA.csv
[CLEANED] KG18A BUKIT BINTANG.csv
[CLEANED] KG20 TRX.csv
[CLEANED] KG21 COCHRANE.csv
[CLEANED] KG22 MALURI.csv
[CLEANED] KG23 TAMAN PERTAMA.csv
[CLEANED] KG24 TAMAN MIDAH.csv
[CLEANED] KG25 TAMAN MUTIARA.csv
[CLEANED] KG27 TAMAN SUNTEX.csv
[CLEANED] KG28 SRI RAYA.csv
[CLEANED] KG29 BTHO.csv
[CLEANED] KG30 BATU 11.csv
[CLEANED] KG31 BUKIT DUKONG.csv
[CLEANED] KG33 SUNGAI JERNIH.csv
[CLEANED] KG34 STADIUM KAJANG.csv
[CLEANED] KG35 KAJANG.csv


In [3]:
#check name 
import os
import sqlite3

# Set your paths
csv_folder = "C:/Users/unitf/OneDrive/Desktop/FYP/mrt_absa_webapp/mrt_reviews_csvs"
db_path = "C:/Users/unitf/OneDrive/Desktop/FYP/mrt_absa_webapp/mrt_reviews.db"

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Fetch station names and normalize them
cursor.execute("SELECT station_name FROM stations")
valid_station_names = set(name.strip().upper() for (name,) in cursor.fetchall())

# Check filenames
all_files = os.listdir(csv_folder)
csv_files = [f for f in all_files if f.endswith(".csv")]

print("Checking filenames...\n")

for file in csv_files:
    station_key = file.replace(".csv", "").strip().upper()
    if station_key in valid_station_names:
        print(f"[OK] Matched: {file}")
    else:
        print(f"[❌] MISMATCH: '{file}' not found in stations table")

conn.close()


Checking filenames...

[OK] Matched: KG04 KWASA DAMANSARA.csv
[OK] Matched: KG06 KOTA DAMANSARA.csv
[OK] Matched: KG07 SURIAN.csv
[OK] Matched: KG08 MUTIARA DAMANSARA.csv
[OK] Matched: KG09 BANDAR UTAMA.csv
[OK] Matched: KG10 TTDI.csv
[OK] Matched: KG11 PHILEO DAMANSARA.csv
[OK] Matched: KG13 PUSAT BANDAR DAMANSARA.csv
[OK] Matched: KG14 SEMANTAN.csv
[OK] Matched: KG15 MUZIUM NEGARA.csv
[OK] Matched: KG16 PASAR SENI.csv
[OK] Matched: KG17 MERDEKA.csv
[OK] Matched: KG18A BUKIT BINTANG.csv
[OK] Matched: KG20 TRX.csv
[OK] Matched: KG21 COCHRANE.csv
[OK] Matched: KG22 MALURI.csv
[OK] Matched: KG23 TAMAN PERTAMA.csv
[OK] Matched: KG24 TAMAN MIDAH.csv
[OK] Matched: KG25 TAMAN MUTIARA.csv
[OK] Matched: KG27 TAMAN SUNTEX.csv
[OK] Matched: KG28 SRI RAYA.csv
[OK] Matched: KG29 BTHO.csv
[OK] Matched: KG30 BATU 11.csv
[OK] Matched: KG31 BUKIT DUKONG.csv
[OK] Matched: KG33 SUNGAI JERNIH.csv
[OK] Matched: KG34 STADIUM KAJANG.csv
[OK] Matched: KG35 KAJANG.csv


In [5]:
import sqlite3

# Connect to your database
conn = sqlite3.connect("C:/Users/unitf/OneDrive/Desktop/FYP/mrt_absa_webapp/mrt_reviews.db")
cursor = conn.cursor()

# Create table
cursor.execute('''
CREATE TABLE IF NOT EXISTS stations (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    station_name TEXT NOT NULL UNIQUE
)
''')

# Add your stations
stations = [
    "KG04 KWASA DAMANSARA", "KG06 KOTA DAMANSARA", "KG07 SURIAN",
    "KG08 MUTIARA DAMANSARA", "KG09 BANDAR UTAMA", "KG10 TTDI",
    "KG11 PHILEO DAMANSARA", "KG13 PUSAT BANDAR DAMANSARA", "KG14 SEMANTAN",
    "KG15 MUZIUM NEGARA", "KG16 PASAR SENI", "KG17 MERDEKA",
    "KG18A BUKIT BINTANG", "KG20 TRX", "KG21 COCHRANE", "KG22 MALURI",
    "KG23 TAMAN PERTAMA", "KG24 TAMAN MIDAH", "KG25 TAMAN MUTIARA",
    "KG27 TAMAN SUNTEX", "KG28 SRI RAYA", "KG29 BTHO", 
    "KG30 BATU 11 CHERAS", "KG31 BUKIT DUKUNG", "KG33 SUNGAI JERNIH",
    "KG34 STADIUM KAJANG", "KG35 KAJANG"
]

for name in stations:
    try:
        cursor.execute("INSERT OR IGNORE INTO stations (station_name) VALUES (?)", (name,))
    except Exception as e:
        print(f"Failed to insert {name}: {e}")

# Commit & close
conn.commit()
conn.close()
print("✅ Station table created and populated.")


OperationalError: database is locked

In [ ]:
import pandas as pd
import re

def clean_text_data(text):
    """
    Cleans a single string by converting to lowercase, removing URLs,
    removing punctuation, and normalizing whitespace.
    """
    if not isinstance(text, str):
        text = str(text) # Handle non-string inputs like NaN

    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

csv_file_name = "KG06 KOTA DAMANSARA.csv" 

df = pd.read_csv(csv_file_name)

original_reviews_column_name = 'reviews'

df['cleaned_reviews'] = df[original_reviews_column_name].apply(clean_text_data)

print("DataFrame with Cleaned Reviews:")
print(df.head())


df.to_csv('KG06 KOTA DAMANSARA.csv', index=False)

In [2]:
#clean data
import pandas as pd
import re

def clean_text_data(text):
    """
    Cleans a single string by converting to lowercase, removing URLs,
    removing punctuation, and normalizing whitespace.
    """
    if not isinstance(text, str):
        text = str(text) # Handle non-string inputs like NaN

    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

csv_file_name = "KG06 kota damansara.csv" 

df = pd.read_csv(csv_file_name)

original_reviews_column_name = 'reviews'

df['cleaned_reviews'] = df[original_reviews_column_name].apply(clean_text_data)

print("DataFrame with Cleaned Reviews:")
print(df.head())


df.to_csv('KG06 KOTA DAMANSARA.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'KG06 kota damansara.csv'